In [1]:
import tensorflow as tf

import numpy as np

2023-12-28 17:45:41.965099: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-28 17:45:41.965142: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-28 17:45:41.966518: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-28 17:45:41.974583: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-28 17:45:42.799413: W tensorflow/compiler/tf2

In [3]:
max_features = 30000
max_len = 200


(X_train, y_train), (X_val, y_val) = tf.keras.datasets.imdb.load_data(num_words=max_features)
print(len(X_train), 'Training sequences')
print(len(X_val), 'Validation sequences')

17464789/17464789 [==============================] - 1s 0us/step
25000 Training sequences
25000 Validation sequences


In [4]:
X_train = tf.keras.utils.pad_sequences(X_train, maxlen=max_len)
X_val = tf.keras.utils.pad_sequences(X_val, maxlen=max_len)

In [5]:
inputs = tf.keras.layers.Input(shape=(None,), dtype='int32')

x = tf.keras.layers.Embedding(max_features, 128)(inputs)

x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))(x)

x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(x)

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs, outputs)

model.summary()

2023-12-28 18:24:12.504094: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-28 18:24:12.530818: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-28 18:24:12.531031: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 128)         3840000   
                                                                 
 bidirectional (Bidirection  (None, None, 128)         98816     
 al)                                                             
                                                                 
 bidirectional_1 (Bidirecti  (None, 128)               98816     
 onal)                                                           
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 4037761 (15.40 MB)
Trainable params: 4037761 (1

In [8]:
model.compile(optimizer='Adam',
             loss='binary_crossentropy',
             metrics=['acc'])

In [9]:
model.fit(X_train, y_train, batch_size=32, epochs=2, validation_data=(X_val, y_val))

Epoch 1/2


2023-12-28 18:27:23.188074: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2023-12-28 18:27:23.549941: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f651426a430 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-28 18:27:23.549964: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1650, Compute Capability 7.5
2023-12-28 18:27:23.555221: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1703798843.621305    5812 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


782/782 [==============================] - 53s 62ms/step - loss: 0.4094 - acc: 0.8132 - val_loss: 0.4425 - val_acc: 0.7753
Epoch 2/2
782/782 [==============================] - 32s 41ms/step - loss: 0.2139 - acc: 0.9178 - val_loss: 0.3672 - val_acc: 0.8456
